<h1>This is a notebook to implement the Final Assignment of IBM Data Science course on Coursera.</h1>

My name is Rafael Fensterseifer and I am from Porto Alegre, Rio Grande do Sul state, Brasil

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h1>1 - First of all, get our data!</h1>

<h3> 1.1 - List of Postal Codes of Toronto (Ontario/Canada) and its latitude and longitude coordinates</h3>

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h4>
    In the next code cell, data was loaded from project resources.<br/>
    The data is a file containing the latitude and longitude coordinates of each postal code area of Toronto. Dataframe <i>dfPostalCodes</i> is join result of that data and dataframe <i>df</i>.
</h4>

In [7]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
dfPostalCodes = df.merge(df_data_1, on='PostalCode', how='left')
dfPostalCodes.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>1.2 - List of Toronto Universities and its latitude and longitude coordinates</h3>

<h4>
    Scrapping the web, I found a page with a list of all universities in Toronto.<br/>
    So I read that and loaded the data into a new dataframe.
</h4>

In [27]:
url = 'http://www.unipage.net/en/universities/toronto'

import requests

r = requests.get(url)
dfUniversities = pd.read_html(r.text)[0]
dfUniversities = dfUniversities[['Title']]
dfUniversities

,Title
0,University of Toronto
1,York University
2,Ryerson University
3,Humber College
4,Centennial College
5,George Brown College
6,OCAD University
7,Michener Institute for Applied Health Sciences
8,Canadian Memorial Chiropractic College
9,Tyndale College & Seminary


In [28]:
dfUniversities.shape

(22, 1)

<h4>
    Using Nominatim, I tried to locate all universities coordinates (latitude and longitude).
</h4>

In [29]:
# The code was removed by Watson Studio for sharing.

In [30]:
# !pip install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [31]:
geolocator = Nominatim(user_agent="foursquare_agent")

In [32]:
def f(row):
    location = geolocator.geocode('{}, Toronto, ON, CA'.format(row.Title))
    if location != None:
        return location.latitude, location.longitude

dfUniversities['coords'] = dfUniversities.apply(f, axis=1)


In [33]:
dfUniversities[['lat','lon']] = pd.DataFrame(dfUniversities['coords'].tolist(), index=dfUniversities.index)
#dfUniversities['lat'], dfUniversities['lon'] = dfUniversities['coords'].apply(lambda c: c if c != None else (np.nan,np.nan))

In [34]:
dfUniversities.drop(columns='coords', inplace=True)
dfUniversities

,Title,lat,lon
0,University of Toronto,43.663462,-79.397760
1,York University,43.779242,-79.483559
2,Ryerson University,43.658469,-79.378993
3,Humber College,43.728512,-79.606250
4,Centennial College,43.785792,-79.227810
5,George Brown College,43.661689,-79.391492
6,OCAD University,43.649334,-79.388887
7,Michener Institute for Applied Health Sciences,NaN,NaN
8,Canadian Memorial Chiropractic College,NaN,NaN
9,Tyndale College & Seminary,NaN,NaN


<h4>
    As you can see above, <i>Nominatin</i> does not found some Universities.<br/>
    <br/>
    Because it is a fundamental data to my analisys, I put those missing values manualy.
</h4>

In [35]:
dfUniversities.iloc[7, [1, 2]] = 43.656068, -79.390157
dfUniversities.iloc[8, [1, 2]] = 43.804010, -79.371692
dfUniversities.iloc[9, [1, 2]] = 43.797215, -79.392058
dfUniversities.iloc[10, [1, 2]] = 43.664140, -79.389595
dfUniversities.iloc[13, [1, 2]] = 43.696793, -79.395161
dfUniversities.iloc[14, [1, 2]] = 43.665362, -79.395217
dfUniversities.iloc[15, [1, 2]] = 43.707839, -79.394081
dfUniversities.iloc[16, [1, 2]] = 43.707852, -79.394303
dfUniversities.iloc[17, [1, 2]] = 43.674188, -79.388516
dfUniversities.iloc[18, [1, 2]] = 43.709327, -79.394502
dfUniversities.iloc[19, [1, 2]] = 43.661566, -79.396597
dfUniversities.iloc[20, [1, 2]] = 43.667066, -79.389729
dfUniversities.iloc[21, [1, 2]] = 43.664792, -79.394359

In [36]:
dfUniversities

,Title,lat,lon
0,University of Toronto,43.663462,-79.397760
1,York University,43.779242,-79.483559
2,Ryerson University,43.658469,-79.378993
3,Humber College,43.728512,-79.606250
4,Centennial College,43.785792,-79.227810
5,George Brown College,43.661689,-79.391492
6,OCAD University,43.649334,-79.388887
7,Michener Institute for Applied Health Sciences,43.656068,-79.390157
8,Canadian Memorial Chiropractic College,43.804010,-79.371692
9,Tyndale College & Seminary,43.797215,-79.392058


<h4>
    Now we can see all locations on a map of Toronto.
</h4>

In [37]:
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # plotting library

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [ ]:
dfUniversitiesClean = dfUniversities.dropna(subset=['lat', 'lon'], axis=0)

In [39]:
latToronto = 43.651070
lonToronto = -79.347015

venues_map = folium.Map(location=[latToronto, lonToronto], zoom_start=11) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latToronto, lonToronto],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lon, label in zip(dfUniversitiesClean.lat, dfUniversitiesClean.lon, dfUniversitiesClean.Title):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
venues_map

<h4>
    Using Geocoder (a system that resolves addresses in CA and US), we can convert the universities coordinates into postal code areas.<br/>
</h4>

In [40]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:011


In [41]:
def getPostalCode(lat, lon):
    neigh = geocoder.geolytica('{}, {}'.format(lat, lon))
    return neigh.json['postal'][:3]

In [42]:
dfUniversitiesClean['PostalCode'] = dfUniversitiesClean.apply(lambda row : getPostalCode(row.lat, row.lon), axis=1)

In [43]:
dfUniversitiesClean

,Title,lat,lon,PostalCode
0,University of Toronto,43.663462,-79.397760,M5S
1,York University,43.779242,-79.483559,M3J
2,Ryerson University,43.658469,-79.378993,M5B
3,Humber College,43.728512,-79.606250,M9W
4,Centennial College,43.785792,-79.227810,M1G
5,George Brown College,43.661689,-79.391492,M7A
6,OCAD University,43.649334,-79.388887,M5V
7,Michener Institute for Applied Health Sciences,43.656068,-79.390157,M5T
8,Canadian Memorial Chiropractic College,43.804010,-79.371692,M2H
9,Tyndale College & Seminary,43.797215,-79.392058,M2M


<h4>
    Now, we can calculate the number of universities per postal code area.
</h4>

In [44]:
dfPostalCodesUniversitiesCount = dfUniversitiesClean[['PostalCode', 'Title']].groupby(['PostalCode']).count().rename(columns={'Title':'Universities'})
dfPostalCodesUniversitiesCount

,Universities
PostalCode,
M1G,1
M2H,1
M2J,1
M2M,1
M3J,1
M4P,3
M4S,1
M4W,1
M5B,1


<h4>
    Finally, we can merge the universities count onto postal codes dataframe.
</h4>

In [45]:
dfPostalCodesUniversities = dfPostalCodes.join(dfPostalCodesUniversitiesCount, on='PostalCode').replace(np.nan, 0.0)
dfPostalCodesUniversities[dfPostalCodesUniversities['Universities'] != 0.0]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Universities
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0
22,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,1.0
52,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,1.0
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3.0
79,M4S,Central Toronto,Davisville,43.704324,-79.388790,1.0
80,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,7.0


In [64]:
dfPostalCodesUniversities.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Universities
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0


<h3>
    1.3 - Using Foursquare API to explore the neighborhoods
</h3>

In [50]:
# The code was removed by Watson Studio for sharing.

In [92]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
nearby_venues = getNearbyVenues(dfPostalCodesUniversities['PostalCode'], 
                                dfPostalCodesUniversities['Latitude'], 
                                dfPostalCodesUniversities['Longitude'])

In [91]:
nearby_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,M4A,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,M4A,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,M4A,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,M5A,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,M5A,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


In [90]:
nearby_venues.shape

(2151, 7)